<a href="https://colab.research.google.com/github/Tanjeelur/Bengali_Summarization/blob/main/Bengali_Summarization_Data_Collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 from google.colab import drive
 drive.mount('/content/drive')

Mounted at /content/drive


# New Section

# New Section

In [ ]:
import pandas as pd
import numpy as np
import os
import re
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer
from transformers import MT5ForConditionalGeneration, MT5Tokenizer

In [ ]:
print(torch.cuda.is_available())


True


In [ ]:
def tokenize_data(df, max_length, max_target_length):
    # Tokenize inputs (source text)
    inputs = tokenizer(
        list(df["text"]),
        max_length=max_length,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )

    # Tokenize targets (summaries) and use them as labels
    targets = tokenizer(
        list(df["summary"]),
        max_length=max_target_length,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )

    # Set the input labels to be the tokenized summaries
    inputs["labels"] = targets["input_ids"]
    return inputs

# Custom Dataset class for tokenized inputs
class TokenizedDataset(Dataset):
    def __init__(self, tokenized_inputs):
        self.input_ids = tokenized_inputs["input_ids"]
        self.attention_mask = tokenized_inputs["attention_mask"]
        self.labels = tokenized_inputs["labels"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "labels": self.labels[idx]
        }

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
source_2_documents_path = "/content/drive/MyDrive/cse498R/BusinessNewsData.csv"
df_2 = pd.read_csv(source_2_documents_path)
df_2.head()

,Title,Description,Meta-Summary,Summary,syndicate-catagorys,Keywords-for-related-articles
0,টাকা ফেরতে ই–অরেঞ্জের লিখিত\r\nপ্রতিশ্রুতি চান...,ই–অরেঞ্জের মালিকপক্ষ যেন বাণিজ্য মন্ত্রণালয়ের ...,ভুক্তভোগী গ্রাহকদের পক্ষে চারজন আজ বুধবার সচিব...,ভুক্তভোগী গ্রাহকদের পক্ষে চারজন আজ বুধবার সচিব...,বাণিজ্য,"ই কমার্স, বাণিজ্য"
1,ক্লিংকার রপ্তানিতে শুল্ক বৃদ্ধির প্রস্তাব ভিয়ে...,সিমেন্ট তৈরির প্রধান কাঁচামাল ক্লিংকার রপ্তানি...,ভিয়েতনামের শুল্ক বাড়ানো হলে ক্লিংকারের দর আরেক...,ভিয়েতনামের শুল্ক বাড়ানো হলে ক্লিংকারের দর আরেক...,বাণিজ্য,"আমদানি রপ্তানি, সিমেন্ট খাত"
2,সূচক ও লেনদেন কমেছে শেয়ারবাজারে,সূচক ও লেনদেন কমেছে দেশের শেয়ারবাজারে। আজ বুধব...,সূচক ও লেনদেন কমেছে দেশের শেয়ারবাজারে। আজ বুধব...,সূচক ও লেনদেন কমেছে দেশের শেয়ারবাজারে। আজ বুধব...,বাণিজ্য,"ডিএসই, সিএসই, ডিএসইএক্স"
3,এমডির চলতি দায়িত্বে শিরীন আখতার,বাংলাদেশ কৃষি ব্যাংকের উপব্যবস্থাপনা পরিচালক শ...,বাংলাদেশ কৃষি ব্যাংকের উপব্যবস্থাপনা পরিচালক শ...,বাংলাদেশ কৃষি ব্যাংকের উপব্যবস্থাপনা পরিচালক শ...,NaN,ব্যাংক
4,মালা শাড়ির আনোয়ার হোসেন,"মালা শাড়ি, বাংলাদেশের শাড়ির জগতে প্রথম সুপরিচি...",দেশের অন্যতম বৃহৎ শিল্পগোষ্ঠী আনোয়ার গ্রুপের চ...,দেশের অন্যতম বৃহৎ শিল্পগোষ্ঠী আনোয়ার গ্রুপের চ...,NaN,"উদ্যোক্তা, ব্যবসায়ী, অর্থনীতি"


In [ ]:
# Use absolute paths
base_path = '/content/drive/MyDrive/bangla'
source_1_documents_path = os.path.join(base_path, 'Documents')
source_1_summaries_path = os.path.join(base_path, 'Summaries')

print(f"Documents path: {source_1_documents_path}")
print(f"Summaries path: {source_1_summaries_path}")

data = []

for filename in os.listdir(source_1_documents_path):
    print(f"Processing file: {filename}")
    if filename.endswith(".txt"):
        document_path = os.path.join(source_1_documents_path, filename)
        print(f"Document path: {document_path}")

        try:
            with open(document_path, 'r', encoding='utf-8', errors='ignore') as doc_file:
                document_content = doc_file.read()

            base_filename = os.path.splitext(filename)[0]
            summary_1_path = os.path.join(source_1_summaries_path, f"{base_filename}_summary_1.txt")
            summary_2_path = os.path.join(source_1_summaries_path, f"{base_filename}_summary_2.txt")

            if os.path.exists(summary_1_path) and os.path.exists(summary_2_path):
                with open(summary_1_path, 'r', encoding='utf-8', errors='ignore') as sum1_file:
                    summary_1_content = sum1_file.read()

                with open(summary_2_path, 'r', encoding='ISO-8859-1', errors='ignore') as sum2_file:
                    summary_2_content = sum2_file.read()

                summary_2_content = re.sub(r'\s+', ' ', summary_2_content).strip()

                data.append({"document": document_content, "summary_1": summary_1_content, "summary_2": summary_2_content})
            else:
                print(f"Skipping {filename}: Summary files not found.")
        except Exception as e:
            print(f"Error processing {filename}: {e}")

df_1 = pd.DataFrame(data)
df_1.head()

Documents path: /content/drive/MyDrive/bangla/Documents
Summaries path: /content/drive/MyDrive/bangla/Summaries
Processing file: document_97.txt
Document path: /content/drive/MyDrive/bangla/Documents/document_97.txt
Processing file: document_98.txt
Document path: /content/drive/MyDrive/bangla/Documents/document_98.txt
Processing file: document_95.txt
Document path: /content/drive/MyDrive/bangla/Documents/document_95.txt
Processing file: document_99.txt
Document path: /content/drive/MyDrive/bangla/Documents/document_99.txt
Processing file: document_96.txt
Document path: /content/drive/MyDrive/bangla/Documents/document_96.txt
Processing file: document_88.txt
Document path: /content/drive/MyDrive/bangla/Documents/document_88.txt
Processing file: document_84.txt
Document path: /content/drive/MyDrive/bangla/Documents/document_84.txt
Processing file: document_83.txt
Document path: /content/drive/MyDrive/bangla/Documents/document_83.txt
Processing file: document_85.txt
Document path: /content

,document,summary_1,summary_2
0,Title:নীলফামারী জেলার সৈয়দপুরে আজকের তাপমাত্রা...,গতকাল রোববার দেশে সর্বনিম্ন তাপমাত্রা ছিল দিনা...,ÿþ¨ À ² « ¾ ® ¾ ° À °  ¸ È ß ¦ ª Á °   ¬ ¹ ¾ ...
1,Title:শেখ রাসেলের ৫৭তম জন্মদিন আজ\n\nText:আজ ব...,আজ বঙ্গবন্ধু শেখ মুজিবুর রহমানের ছোট ছেলে শেখ ...,ÿþ   ¬  Í  ¬ ¨ Í § Á  ¶ Ç   ® Á  ¿ ¬ Á °...
2,Title:খালেদা জিয়ার সাজার রায়ঃ ৫ বছরের জেল/কারা...,সরকারি টাকা আত্মসাতের মামলায় বিএনপি চেয়ারপারসন...,ÿþ¸ °  ¾ ° ¿   ¾  ¾   ¤ Í ® ¸ ¾ ¤ Ç °  ® ¾...
3,Title:মানসিক স্বাস্থ্যসেবা নিশ্চিতের আহ্বান সা...,টেকসই উন্নয়ন লক্ষ্যমাত্রা অর্জনে মানসিক স্বাস্...,ÿþ  ª Í ° ¤ ¿ ¬ ¨ Í § À  ¬ Í ¯  Í ¤ ¿  ¬ ¾  ®...
4,Title:ঝুলন্ত তার ভূগর্ভে নেয়ার উদ্যোগ \n\nText...,"ডিসের লাইন, ইন্টারনেট লাইন, টেলিফোন লাইনসহ যত্...",ÿþ Á ² ¨ Í ¤  ¤ ¾ °  ¸ ° ¿ ß Ç  ¶ ¹ ° Ç °  ¸ ...


In [ ]:
print(df_1.columns)

Index(['document', 'summary_1', 'summary_2'], dtype='object')


In [ ]:
# Define file paths
source_3_articles_path = "/content/drive/MyDrive/bangla/Bengali-News-Summarization-Dataset/article.txt"
source_3_summaries_path = "/content/drive/MyDrive/bangla/Bengali-News-Summarization-Dataset/summary.txt"

# Read the files and split by lines, assuming each line is an article/summary pair
with open(source_3_articles_path, 'r', encoding='utf-8') as file:
    articles = file.readlines()

with open(source_3_summaries_path, 'r', encoding='utf-8') as file:
    summaries = file.readlines()

# Ensure both lists have the same number of entries
if len(articles) != len(summaries):
    raise ValueError("The number of articles and summaries do not match!")

# Create a DataFrame with each article-summary pair in one row
data = {"Article": [article.strip() for article in articles],
        "Summary": [summary.strip() for summary in summaries]}

df_3 = pd.DataFrame(data)

# Display the DataFrame
df_3.head()  # Use df to see the entire DataFrame

,Article,Summary
0,স্ট্যান্ডার্ড চার্টার্ড ব্যাংকের নতুন প্রধান ন...,স্ট্যান্ডার্ড চার্টার্ডের নতুন সিইও আবরার
1,রাজধানী থেকে চামড়া শিল্পগুলো সাভারে স্থানান্তর...,মার্চের মধ্যে সাভারে চামড়া শিল্পের সিইটিপি: মন...
2,দেশীয় শিল্প বিকাশে সরকারের সব ধরনের উদ্যোগ অব্...,ওয়ালটন কারখানায় শিল্পমন্ত্রী
3,একীভূত হতে চলেছে অনলাইনে শ্রেণিবদ্ধ বিজ্ঞাপন স...,একীভূত হচ্ছে এখানেই ডটকমওএলএক্স
4,যাত্রীবাহী একটি বাসে আগুন দেওয়ার আধা ঘণ্টার মধ...,বাসে আগুন: নড়াইলের পৌর মেয়র গ্রেপ্তার


In [ ]:
# Define file paths
source_4_test_path = '/content/drive/MyDrive/bangla/test.jsonl'
source_4_train_path = '/content/drive/MyDrive/bangla/train.jsonl'
source_4_val_path = '/content/drive/MyDrive/bangla/val.jsonl'

# Load each JSONL file into a DataFrame
df_4_test = pd.read_json(source_4_test_path, lines=True)
df_4_train = pd.read_json(source_4_train_path, lines=True)
df_4_val = pd.read_json(source_4_val_path, lines=True)
# changing the coloumn name
df_4_test.rename(columns={'Title': 'text'}, inplace=True)
df_4_train.rename(columns={'Title': 'text'}, inplace=True)
df_4_val.rename(columns={'Title': 'text'}, inplace=True)

df_4_test.rename(columns={'Summary': 'summary'}, inplace=True)
df_4_train.rename(columns={'Summary': 'summary'}, inplace=True)
df_4_val.rename(columns={'Summary': 'summary'}, inplace=True)

In [ ]:
# # Rename the 'Title' column to 'text'
# df_4_test.rename(columns={'Title': 'text'}, inplace=True)
# df_4_train.rename(columns={'Title': 'text'}, inplace=True)
# df_4_val.rename(columns={'Title': 'text'}, inplace=True)


NameError: name 'df_4' is not defined

In [ ]:
df_4_test.head()

,text,summary
0,আগ্নেয়াস্ত্র নিয়ন্ত্রণে নির্বাহী আদেশ দিতে যাচ...,যুক্তরাষ্ট্রে আগ্নেয়াস্ত্র নিয়ন্ত্রণে গুরুত্বপ...
1,খুলনার চার হাসপাতালে সর্বোচ্চ ২২ জনের মৃত্যু,গতকাল মঙ্গলবার থেকে আজ বুধবার পর্যন্ত ২৪ ঘণ্টা...
2,"সীতাকুণ্ডে গৃহবধূর লাশ উদ্ধার, স্বামী পলাতক",আজ বৃহস্পতিবার বেলা তিনটার দিকে পুলিশ সোনিয়ার ...
3,ক্রিকেটারদের পরামর্শও দিতে পারবেন না সৌরভ?,শ্রেয়াস আইয়ারকে পরামর্শ দিয়ে সমালোচকদের তোপের ...
4,মাকে নিয়ে বাড়ি ফেরা হলো না মাহফুজুরের,কাভার্ড ভ্যানের ধাক্কায় মা ও ছেলে দুজনই মহাসড়ক...


In [ ]:
df_4_train.head()

,text,Summary
0,এসেছে চরকির প্রথম অরিজিনাল ওয়েব সিরিজ ‘মরীচিকা...,সিরিজটির গল্প এক তরুণ মডেলের হত্যার তদন্ত নিয়ে...
1,রিটার্ন জমার সময় বাড়ল ১ মাস,অবশেষে আয়কর রিটার্ন দাখিলের সময় বাড়ল ১ মাস। ৩১...
2,ফুডপান্ডার পেমেন্ট এখন বিকাশে,করোনা মহামারিতে নগদ অর্থের লেনদেনে যে স্বাস্থ্...
3,জামিন পেলেন ‘নবাব এলএলবি’র পরিচালক ও অভিনেতা,চলচ্চিত্র পরিচালক অনন্য মামুন ও অভিনেতা শাহীন ...
4,হতাশার বৃত্ত থেকে বেরোতেই পারছে না জার্মানি,উয়েফা নেশনস লিগের ম্যাচে আবারও হোঁচট খেয়েছে জা...


In [ ]:
df_4_val.head()

,text,Summary
0,"নারদা মামলা এবার সুপ্রিম কোর্টে, আজ শুনানি",নারদা মামলার শুনানি হচ্ছিল কলকাতা হাইকোর্টের প...
1,"বরিশাল বিভাগে ২৪ ঘণ্টায় মৃত্যু ২০, শনাক্ত ৮২২",ঈদের কারণে নমুনা সংগ্রহ কম হওয়ায় শনাক্তও কম ছি...
2,আবার নাসুমের আঘাত,শুরুতেই নিউজিল্যান্ডকে ধাক্কা দিতে পেরেছে বাংল...
3,আত্মহত্যা করেছেন জাপানের জনপ্রিয় তারকা,জাপানের জনপ্রিয় এই শিল্পী অনেক সিনেমা ও টেলিভি...
4,ব্রিটিশ যুদ্ধজাহাজকে সতর্ক করতে রাশিয়ার গুলি ও...,রাশিয়ার যুদ্ধবিমান ও জাহাজ থেকে যুক্তরাজ্যের য...


In [ ]:
# Define the root directories
source_5_bnlpc_dataset1_root = '/content/drive/MyDrive/bangla/BNLPC/Dataset1'
source_5_bnlpc_dataset2_root = '/content/drive/MyDrive/bangla/BNLPC/Dataset2'
source_5_nctb_root = '/content/drive/MyDrive/bangla/NCTB'

# Function to load articles, summaries, normalized documents, and titles
def load_bnlpc_dataset(documents_path, normalize_document_path, normalize_title_path, summaries_path):
    documents = []
    normalized_documents = []
    normalized_titles = []
    summaries = []

    # Get all files (assumes filenames match for each pair)
    doc_files = sorted(os.listdir(documents_path))
    norm_doc_files = sorted(os.listdir(normalize_document_path))
    norm_title_files = sorted(os.listdir(normalize_title_path))
    summary_files = sorted(os.listdir(summaries_path))

    for doc_file, norm_doc_file, norm_title_file, summary_file in zip(doc_files, norm_doc_files, norm_title_files, summary_files):
        # Read the document
        with open(os.path.join(documents_path, doc_file), 'r', encoding='utf-8') as f:
            documents.append(f.read().strip())

        # Read the normalized document
        with open(os.path.join(normalize_document_path, norm_doc_file), 'r', encoding='utf-8') as f:
            normalized_documents.append(f.read().strip())

        # Read the normalized title
        with open(os.path.join(normalize_title_path, norm_title_file), 'r', encoding='utf-8') as f:
            normalized_titles.append(f.read().strip())

        # Read the summary
        with open(os.path.join(summaries_path, summary_file), 'r', encoding='utf-8') as f:
            summaries.append(f.read().strip())

    # Create a DataFrame with the documents, normalized documents, titles, and summaries
    return pd.DataFrame({
        'Document': documents,
        'NormalizeDocument': normalized_documents,
        'NormalizeTitle': normalized_titles,
        'Summary': summaries
    })

# Load BNLPC Dataset1
bnlpc_dataset1_documents_path = os.path.join(source_5_bnlpc_dataset1_root, 'Documents')
bnlpc_dataset1_normalize_document_path = os.path.join(source_5_bnlpc_dataset1_root, 'NormaliseDocument')
bnlpc_dataset1_normalize_title_path = os.path.join(source_5_bnlpc_dataset1_root, 'NormaliseTitle')
bnlpc_dataset1_summaries_path = os.path.join(source_5_bnlpc_dataset1_root, 'Summaries')

df_5_bnlpc_dataset1 = load_bnlpc_dataset(
    bnlpc_dataset1_documents_path,
    bnlpc_dataset1_normalize_document_path,
    bnlpc_dataset1_normalize_title_path,
    bnlpc_dataset1_summaries_path
)

# Load BNLPC Dataset2
bnlpc_dataset2_documents_path = os.path.join(source_5_bnlpc_dataset2_root, 'Documents')
bnlpc_dataset2_normalize_document_path = os.path.join(source_5_bnlpc_dataset2_root, 'NormaliseDocument')
bnlpc_dataset2_normalize_title_path = os.path.join(source_5_bnlpc_dataset2_root, 'NormaliseTitle')
bnlpc_dataset2_summaries_path = os.path.join(source_5_bnlpc_dataset2_root, 'Summaries')

df_5_bnlpc_dataset2 = load_bnlpc_dataset(
    bnlpc_dataset2_documents_path,
    bnlpc_dataset2_normalize_document_path,
    bnlpc_dataset2_normalize_title_path,
    bnlpc_dataset2_summaries_path
)

# Function to load NCTB dataset (just source and summary)
def load_nctb_dataset(source_path, summary_path):
    sources = []
    summaries = []

    source_files = sorted(os.listdir(source_path))
    summary_files = sorted(os.listdir(summary_path))

    for source_file, summary_file in zip(source_files, summary_files):
        # Read the source document
        with open(os.path.join(source_path, source_file), 'r', encoding='utf-8') as f:
            sources.append(f.read().strip())

        # Read the summary
        with open(os.path.join(summary_path, summary_file), 'r', encoding='utf-8') as f:
            summaries.append(f.read().strip())

    # Create a DataFrame with the source and summary
    return pd.DataFrame({
        'Source': sources,
        'Summary': summaries
    })

# Load NCTB dataset
nctb_source_path = os.path.join(source_5_nctb_root, 'Source')
nctb_summary_path = os.path.join(source_5_nctb_root, 'Summary')
df_5_nctb = load_nctb_dataset(nctb_source_path, nctb_summary_path)

In [ ]:
df_5_bnlpc_dataset1.head()

,Document,NormalizeDocument,NormalizeTitle,Summary
0,Title: লতিফ সিদ্দিকী ছয় মাসের জামিন \n\nText: ...,ধর্মীয় অনুভূতিতে আঘাতের অভিযোগে করা আরও ১০ মাম...,লতিফ সিদ্দিকী ছয় মাসের জামিন,উচ্চ আদালতে নির্দোষ প্রমাণিত হওয়ার পর গতকাল ব...
1,Title: \nযে শর্তে অজয়ের ঘরনী হয়েছিলেন কাজল \n\...,জনপ্রিয়তার তুঙ্গে থাকার সময় ১৯৯৯ সালে কাজল বিয়...,যে শর্তে অজয়ের ঘরনী হয়েছিলেন কাজল,উচ্চ আদালতে নির্দোষ প্রমাণিত হওয়ার পর গতকাল ব...
2,Title: \nখালাসের রায়ের ১৩ বছর পর কারামুক্ত হলে...,উচ্চ আদালতে নির্দোষ প্রমাণিত হওয়ার পর গতকাল ব...,খালাসের রায়ের ১৩ বছর পর কারামুক্ত হলেন জবেদ আলী,উচ্চ আদালতে নির্দোষ প্রমাণিত হওয়ার পর গতকাল ব...
3,Title: \nরংপুরে মানহানির মামলায় মাহফুজ আনামের ...,রংপুরে ১০০ কোটি টাকার মানহানির মামলায় ডেইলি স্...,রংপুরে মানহানির মামলায় মাহফুজ আনামের জামিন মঞ্জুর,জনপ্রিয়তার তুঙ্গে থাকার সময় ১৯৯৯ সালে কাজল বিয়...
4,Title: \nসাত মামলায় জামিন মিন্টুর \n\nText: \...,গাড়ি ভাঙচুর ও নাশকতার অভিযোগে পুলিশের করা সাত...,সাত মামলায় জামিন মিন্টুর,জনপ্রিয়তার তুঙ্গে থাকার সময় ১৯৯৯ সালে কাজল বিয়...


In [ ]:
df_5_bnlpc_dataset2.head()

,Document,NormalizeDocument,NormalizeTitle,Summary
0,"Title: সোনারগাঁয়ে মহাসড়কে যুবলীগের সভা, দীর্...",ঢাকা-চট্টগ্রাম মহাসড়কের ওপর মঞ্চ বানিয়ে গতকাল...,"সোনারগাঁয়ে মহাসড়কে যুবলীগের সভা, দীর্ঘ যানজট...",খালেদা জিয়ার বিরুদ্ধে মামলা দিয়ে সরকার বিএনপির...
1,Title: \nট্যাংকে পিষ্ট বিদ্রোহ-প্রচেষ্টা \n\n...,আল্লাহর নামে আর অখণ্ড পাকিস্তান রক্ষার অজুহাতে...,ট্যাংকে পিষ্ট বিদ্রোহ-প্রচেষ্টা,খালেদা জিয়ার বিরুদ্ধে মামলা দিয়ে সরকার বিএনপির...
2,Title: \nসরকার বিএনপির কাউন্সিল ‘ভন্ডুলের ষড়যন...,খালেদা জিয়ার বিরুদ্ধে মামলা দিয়ে সরকার বিএনপির...,সরকার বিএনপির কাউন্সিল ‘ভন্ডুলের ষড়যন্ত্রে লিপ্ত’,"তিনি বলেন, \""সরকার বিএনপির চেয়ারপারসন খালেদা জ..."
3,Title: \nবেআইনি প্লট ফেরতে দায়মুক্তি \n\nText:...,ইংল্যান্ডের প্রাচীন যুগের নৃপতিরা ব্যক্তিবিশেষ...,বেআইনি প্লট ফেরতে দায়মুক্তি,আল্লাহর নামে আর অখণ্ড পাকিস্তান রক্ষার অজুহাতে...
4,Title: \nগুণীজন ও কৃতী শিক্ষার্থীদের সংবর্ধনা ...,সৌদি আরবের রিয়াদে প্রবাসী বাংলাদেশি গুণীজন ও ক...,গুণীজন ও কৃতী শিক্ষার্থীদের সংবর্ধনা,ঠান্ডা মাথায় পাকিস্তানি সৈন্যরা টানা ২৪ ঘণ্টা ...


In [ ]:
df_5_nctb.head()

,Source,Summary
0,মানুষের সুন্দর মুখ দেখে আনন্দিত হয়ো না। স্বভাব...,"বাহ্যিক সৌন্দর্য নয়, স্বভাবের সৌন্দর্যই মানুষক..."
1,"মুখে অনেকেই টাকা তুচ্ছ, অর্থ অনর্থের মূল বলিয়া...",অর্থকে অনর্থের মূল বলা হলেও মূলত এটি অত্যন্ত ম...
2,"আমি ভাগিনীদের কল্যাণ কামনা করি, তাহাদের ধর্মবন...",উপযুক্ত শিক্ষার অভাবে নারী তার মনের স্বাধীনতা ...
3,"আমি ভালোবেসেছি এ জগৎকে, আমি প্রণাম করেছি মহৎকে...",একজন জীবনসাধকের কাছে এ জগতই সত্য। তিনি মানুষের...
4,আমি লাইব্রেরিকে স্কুল কলেজের উপরে স্থান দিই এই...,স্বশিক্ষিত হবার সুযোগ আমাদের প্রচলিত শিক্ষা ব্...


In [ ]:
# Initialize tokenizer (replace with your desired model's tokenizer)
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

# Function to calculate descriptive statistics for token length
def descriptive_statistics_tokens(df, column_name):
    # Calculate length of each entry in terms of tokens
    df[f"{column_name}_token_length"] = df[column_name].apply(lambda x: len(tokenizer.tokenize(x)))

    # Calculate descriptive statistics
    stats = {
        "mean_length": df[f"{column_name}_token_length"].mean(),
        "max_length": df[f"{column_name}_token_length"].max(),
        "min_length": df[f"{column_name}_token_length"].min(),
        "std_length": df[f"{column_name}_token_length"].std()
    }
    return stats

# Calculate statistics for text and summary columns in each dataset
train_text_stats = descriptive_statistics_tokens(df_4_train, "text")
train_summary_stats = descriptive_statistics_tokens(df_4_train, "summary")

val_text_stats = descriptive_statistics_tokens(df_4_val, "text")
val_summary_stats = descriptive_statistics_tokens(df_4_val, "summary")

test_text_stats = descriptive_statistics_tokens(df_4_test, "text")
test_summary_stats = descriptive_statistics_tokens(df_4_test, "summary")

# Display the statistics
print("Training Set - Text (token count):", train_text_stats)
print("Training Set - Summary (token count):", train_summary_stats)

print("\nValidation Set - Text (token count):", val_text_stats)
print("Validation Set - Summary (token count):", val_summary_stats)

print("\nTest Set - Text (token count):", test_text_stats)
print("Test Set - Summary (token count):", test_summary_stats)

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Training Set - Text (token count): {'mean_length': 18.151728553137005, 'max_length': 41, 'min_length': 4, 'std_length': 4.82557346981042}
Training Set - Summary (token count): {'mean_length': 74.8031903542467, 'max_length': 247, 'min_length': 2, 'std_length': 31.22861384775409}

Validation Set - Text (token count): {'mean_length': 18.161758429364063, 'max_length': 37, 'min_length': 5, 'std_length': 4.731099461629275}
Validation Set - Summary (token count): {'mean_length': 76.18139137857447, 'max_length': 223, 'min_length': 4, 'std_length': 31.05805635422443}

Test Set - Text (token count): {'mean_length': 18.075511945392492, 'max_length': 38, 'min_length': 5, 'std_length': 4.715627907088927}
Test Set - Summary (token count): {'mean_length': 75.06996587030717, 'max_length': 214, 'min_length': 2, 'std_length': 31.810958134682956}


In [ ]:
max_length = 512
max_target_length = 100

In [ ]:
model_name = "google/mt5-small"  # Pretrained mT5-small model
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'MT5Tokenizer'.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

# Tokenize the datasets with labels
train_inputs = tokenize_data(df_4_train, max_length=512, max_target_length=100)
val_inputs = tokenize_data(df_4_val, max_length=512, max_target_length=100)
test_inputs = tokenize_data(df_4_test, max_length=512, max_target_length=100)

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
# Create datasets from tokenized inputs
train_dataset = TokenizedDataset(train_inputs)
val_dataset = TokenizedDataset(val_inputs)
test_dataset = TokenizedDataset(test_inputs)

In [ ]:
# Define Seq2SeqTrainingArguments for generation tasks
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    predict_with_generate=True,  # Enables generation during evaluation
    save_safetensors=False  # Disable saving in safetensors format
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Define Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer  # Pass the tokenizer for decoding during evaluation
)

<ipython-input-55-4a15afb888f8>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: md-tanjeelur (md-tanjeelur-north-south-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,4.375500,2.625609
2,3.109400,2.217479
3,2.956200,2.175135


TrainOutput(global_step=7029, training_loss=5.202328415883013, metrics={'train_runtime': 6174.9925, 'train_samples_per_second': 9.106, 'train_steps_per_second': 1.138, 'total_flos': 2.973265261756416e+16, 'train_loss': 5.202328415883013, 'epoch': 3.0})

In [ ]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 2.175135374069214, 'eval_runtime': 49.5432, 'eval_samples_per_second': 47.292, 'eval_steps_per_second': 5.914, 'epoch': 3.0}


In [ ]:
# Evaluate on the test dataset
eval_results = trainer.evaluate(eval_dataset=test_dataset)
print(eval_results)

{'eval_loss': 2.145789861679077, 'eval_runtime': 49.5216, 'eval_samples_per_second': 47.333, 'eval_steps_per_second': 5.917, 'epoch': 3.0}


In [ ]:
df_4_test.head()

,text,summary,text_token_length,summary_token_length
0,আগ্নেয়াস্ত্র নিয়ন্ত্রণে নির্বাহী আদেশ দিতে যাচ...,যুক্তরাষ্ট্রে আগ্নেয়াস্ত্র নিয়ন্ত্রণে গুরুত্বপ...,26,74
1,খুলনার চার হাসপাতালে সর্বোচ্চ ২২ জনের মৃত্যু,গতকাল মঙ্গলবার থেকে আজ বুধবার পর্যন্ত ২৪ ঘণ্টা...,16,50
2,"সীতাকুণ্ডে গৃহবধূর লাশ উদ্ধার, স্বামী পলাতক",আজ বৃহস্পতিবার বেলা তিনটার দিকে পুলিশ সোনিয়ার ...,21,81
3,ক্রিকেটারদের পরামর্শও দিতে পারবেন না সৌরভ?,শ্রেয়াস আইয়ারকে পরামর্শ দিয়ে সমালোচকদের তোপের ...,17,67
4,মাকে নিয়ে বাড়ি ফেরা হলো না মাহফুজুরের,কাভার্ড ভ্যানের ধাক্কায় মা ও ছেলে দুজনই মহাসড়ক...,16,59


In [ ]:
sample_text = df_4_test.iloc[0]['text']
print(type(sample_text))

<class 'str'>


In [ ]:
# Make sure the input_ids tensor is on the same device as the model
device = model.device  # Get the device of the model (e.g., 'cuda:0' or 'cpu')
input_ids = tokenizer.encode(sample_text, return_tensors="pt", max_length=512, truncation=True).to(device)

# Generate summary
outputs = model.generate(input_ids, max_length=100, num_beams=5, early_stopping=True)
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(summary)

ভারতীয় প্রেসিডেন্ট বাইডেন বাইডেন।


In [ ]:
df_4_test.iloc[0]['summary']

'যুক্তরাষ্ট্রে আগ্নেয়াস্ত্র নিয়ন্ত্রণে গুরুত্বপূর্ণ নির্বাহী আদেশ জারি করতে যাচ্ছেন প্রেসিডেন্ট জো বাইডেন। স্থানীয় সময় বৃহস্পতিবার হোয়াইট হাউস থেকে বাইডেন এ-সংক্রান্ত প্রথম পদক্ষেপের জানান দেবেন'

In [ ]:
trainer.train()

NameError: name 'trainer' is not defined

In [ ]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 1.409576654434204, 'eval_runtime': 23.0218, 'eval_samples_per_second': 43.958, 'eval_steps_per_second': 5.517, 'epoch': 3.0}


In [ ]:
# Evaluate on the test dataset
eval_results = trainer.evaluate(eval_dataset=test_dataset)
print(eval_results)

{'eval_loss': 1.4490957260131836, 'eval_runtime': 23.1836, 'eval_samples_per_second': 43.651, 'eval_steps_per_second': 5.478, 'epoch': 3.0}


In [ ]:
sample_text = df_4_test.iloc[5]['text']
print(sample_text)

মাওবাদী এলাকায় পুলিশের টহল (ফাইল ফটো) ভারতে যে রাজ্যগুলোতে মাওবাদীদের দাপট বেশি, সেগুলোর মুখ্যমন্ত্রী ও পুলিশ-প্রধানদের সঙ্গে বৈঠকেই তিনি সোমবার এ মন্তব্য করেন। মাওবাদীদের সঙ্গে শান্তি-আলোচনার কথা যে সরকার আদৌ ভাবছে না, সেটাও তিনি স্পষ্ট করে দিয়েছেন। দুসপ্তাহ আগে ছত্তিশগড়ের সুকমায় মাওবাদীদের হামলায় ২৫জন সিআরপিএফ জওয়ান নিহত হওয়ার পরই ভারতের কেন্দ্রীয় সরকার তড়িঘড়ি আজকের এই বৈঠক ডেকেছিল। সেখানে স্বরাষ্ট্রমন্ত্রী রাজনাথ সিং স্পষ্ট করে দেন মাওবাদীদের দমনে সরকারি কৌশলে বিরাট কোনও পরিবর্তন হচ্ছে না। তিনি বলেন, "সিলভার বুলেট দিয়ে এই সমস্যা মিটবে না - অর্থাৎ ঝটিতি কোনও জাদুবলে এর সমাধান মিলবে না, আর এর কোনও শর্টকাটও নেই। স্বল্প, মাঝারি ও দীর্ঘমেয়াদে নানা ধরনের হাতিয়ার ব্যবহার করেই আমাদের এগোতে হবে।" ইংরেজিতে সমাধান শব্দের এস, এ, এম ইত্যাদি সবগুলো বর্ণের জন্য স্মার্ট লিডারশিপ, অ্যাগ্রেসিভ স্ট্র্যাটেজি, মোটিভেশন এজাতীয় নানা শব্দ ব্যবহার করে স্বরাষ্ট্রমন্ত্রী দাবি করেন, এই সমাধান শব্দেই লুকিয়ে আছে মাওবাদীদের জবাব। আরও পড়ুন: নির্বাচনের পালে হাওয়া, নেই আগাম ভোটের ইঙ্গিত কিভাবে সনাক্

In [ ]:
# Make sure the input_ids tensor is on the same device as the model
device = model.device  # Get the device of the model (e.g., 'cuda:0' or 'cpu')
input_ids = tokenizer.encode(sample_text, return_tensors="pt", max_length=512, truncation=True).to(device)

# Generate summary
outputs = model.generate(input_ids, max_length=200, num_beams=7, early_stopping=True)
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(summary)

বাংলাদেশে মাওবাদীদের হামলায় ২৫জন সিআরপিএফ জওয়ান নিহত হওয়ার পর ভারতের কেন্দ্রীয় সরকার তড়িঘড়ি আজকের এই বৈঠক ডেকেছে।


In [ ]:
df_4_test.iloc[5]['summary']

'ভারতে মাওবাদী বিদ্রোহীদের মোকাবিলার পথ খুঁজতে সরকারের এক উচ্চ পর্যায়ের বৈঠকের পর স্বরাষ্ট্রমন্ত্রী রাজনাথ সিং জানিয়েছেন, এই সঙ্কটের কোনও চটজলদি সমাধান সম্ভব নয়, সর্বাত্মক উন্নয়ন আর আগ্রাসী নীতি বজায় রেখেই মাওবাদীদের কাবু করা সম্ভব।'

In [ ]:
# Ensure the model is on CPU
model.to("cpu")

# Make sure the input_ids tensor is on the same device as the model (CPU in this case)
device = model.device  # This should now be 'cpu'
input_ids = tokenizer.encode(sample_text, return_tensors="pt", max_length=1024, truncation=True).to(device)

# Generate summary
outputs = model.generate(input_ids, max_length=200, num_beams=7, early_stopping=True)
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(summary)

বাংলাদেশে মাওবাদী বিদ্রোহ দমনের সহজ সমাধান নেই।


In [ ]:
save_path = '/content/drive/MyDrive/bangla_text_summarizer/mt5_weights/'

# Make all model parameters contiguous
for param in model.parameters():
    if not param.is_contiguous():
        param.data = param.data.contiguous()

# Save the model and tokenizer
model.save_pretrained(f"{save_path}fine_tuned_mt5_summarization")
tokenizer.save_pretrained(f"{save_path}fine_tuned_mt5_summarization")

('/content/drive/MyDrive/bangla_text_summarizer/mt5_weights/fine_tuned_mt5_summarization/tokenizer_config.json',
 '/content/drive/MyDrive/bangla_text_summarizer/mt5_weights/fine_tuned_mt5_summarization/special_tokens_map.json',
 '/content/drive/MyDrive/bangla_text_summarizer/mt5_weights/fine_tuned_mt5_summarization/spiece.model',
 '/content/drive/MyDrive/bangla_text_summarizer/mt5_weights/fine_tuned_mt5_summarization/added_tokens.json',
 '/content/drive/MyDrive/bangla_text_summarizer/mt5_weights/fine_tuned_mt5_summarization/tokenizer.json')

In [ ]:
max_length = 512
max_target_length = 200

In [ ]:
# Define the path to your saved model and tokenizer
load_path = '/content/drive/MyDrive/bangla_text_summarizer/mt5_weights/fine_tuned_mt5_summarization_v3'

# Load the tokenizer
tokenizer = MT5Tokenizer.from_pretrained(load_path)

# Load the model
model = MT5ForConditionalGeneration.from_pretrained(load_path)

# Tokenize the datasets with labels
train_inputs = tokenize_data(df_4_train, max_length, max_target_length)
val_inputs = tokenize_data(df_4_val,  max_length, max_target_length)
test_inputs = tokenize_data(df_4_test,  max_length, max_target_length)

In [ ]:
# Create datasets from tokenized inputs
train_dataset = TokenizedDataset(train_inputs)
val_dataset = TokenizedDataset(val_inputs)
test_dataset = TokenizedDataset(test_inputs)

In [ ]:
# Define Seq2SeqTrainingArguments for generation tasks
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=2,
    predict_with_generate=True,  # Enables generation during evaluation
    save_safetensors=False  # Disable saving in safetensors format
)

In [ ]:
# Define Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer  # Pass the tokenizer for decoding during evaluation
)

In [ ]:
model.device

device(type='cuda', index=0)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,1.046200,0.692979
2,1.015400,0.683604
3,1.027700,0.676918
4,0.988000,0.672858
5,0.994400,0.671896


TrainOutput(global_step=5065, training_loss=1.0143020765449546, metrics={'train_runtime': 5081.3084, 'train_samples_per_second': 7.972, 'train_steps_per_second': 0.997, 'total_flos': 2.14196499775488e+16, 'train_loss': 1.0143020765449546, 'epoch': 5.0})

In [ ]:
sample_text = df_4_test.iloc[5]['text']
print(sample_text)

মাওবাদী এলাকায় পুলিশের টহল (ফাইল ফটো) ভারতে যে রাজ্যগুলোতে মাওবাদীদের দাপট বেশি, সেগুলোর মুখ্যমন্ত্রী ও পুলিশ-প্রধানদের সঙ্গে বৈঠকেই তিনি সোমবার এ মন্তব্য করেন। মাওবাদীদের সঙ্গে শান্তি-আলোচনার কথা যে সরকার আদৌ ভাবছে না, সেটাও তিনি স্পষ্ট করে দিয়েছেন। দুসপ্তাহ আগে ছত্তিশগড়ের সুকমায় মাওবাদীদের হামলায় ২৫জন সিআরপিএফ জওয়ান নিহত হওয়ার পরই ভারতের কেন্দ্রীয় সরকার তড়িঘড়ি আজকের এই বৈঠক ডেকেছিল। সেখানে স্বরাষ্ট্রমন্ত্রী রাজনাথ সিং স্পষ্ট করে দেন মাওবাদীদের দমনে সরকারি কৌশলে বিরাট কোনও পরিবর্তন হচ্ছে না। তিনি বলেন, "সিলভার বুলেট দিয়ে এই সমস্যা মিটবে না - অর্থাৎ ঝটিতি কোনও জাদুবলে এর সমাধান মিলবে না, আর এর কোনও শর্টকাটও নেই। স্বল্প, মাঝারি ও দীর্ঘমেয়াদে নানা ধরনের হাতিয়ার ব্যবহার করেই আমাদের এগোতে হবে।" ইংরেজিতে সমাধান শব্দের এস, এ, এম ইত্যাদি সবগুলো বর্ণের জন্য স্মার্ট লিডারশিপ, অ্যাগ্রেসিভ স্ট্র্যাটেজি, মোটিভেশন এজাতীয় নানা শব্দ ব্যবহার করে স্বরাষ্ট্রমন্ত্রী দাবি করেন, এই সমাধান শব্দেই লুকিয়ে আছে মাওবাদীদের জবাব। আরও পড়ুন: নির্বাচনের পালে হাওয়া, নেই আগাম ভোটের ইঙ্গিত কিভাবে সনাক্

In [ ]:
# Make sure the input_ids tensor is on the same device as the model
device = model.device  # Get the device of the model (e.g., 'cuda:0' or 'cpu')
input_ids = tokenizer.encode(sample_text, return_tensors="pt", max_length=512, truncation=True).to(device)

# Generate summary
outputs = model.generate(input_ids, max_length=200, num_beams=7, early_stopping=True)
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(summary)

বাংলাদেশে মাওবাদীদের হামলায় ২৫জন সিআরপিএফ জওয়ান নিহত হওয়ার পর ভারতের কেন্দ্রীয় সরকার তড়িঘড়ি আজকের বৈঠক ডেকেছে।


In [ ]:
df_4_test.iloc[5]['summary']

'ভারতে মাওবাদী বিদ্রোহীদের মোকাবিলার পথ খুঁজতে সরকারের এক উচ্চ পর্যায়ের বৈঠকের পর স্বরাষ্ট্রমন্ত্রী রাজনাথ সিং জানিয়েছেন, এই সঙ্কটের কোনও চটজলদি সমাধান সম্ভব নয়, সর্বাত্মক উন্নয়ন আর আগ্রাসী নীতি বজায় রেখেই মাওবাদীদের কাবু করা সম্ভব।'

In [ ]:
sample_text = """‘ইকোনমিস্ট’ লিখেছে, অ্যান্টিবডির চার মাস স্থায়ী হওয়ার খবরটি দুই কারণে আনন্দের। অ্যান্টিবডি যত দিন পর্যন্ত শরীরে টিকবে, তত দিন সংক্রমণ থেকে সুরক্ষিত থাকা সম্ভব। অর্থাৎ, এমন এক টিকার প্রয়োজন হবে, যা অ্যান্টিবডির উত্পাদনকে প্ররোচিত করতে পারে এবং দীর্ঘস্থায়ী সুরক্ষা দিতে পারে। এগুলো খুঁজে বের করাও সহজ। এটি আভাস দেয়, ব্যাপক হারে অ্যান্টিবডি শনাক্তকরণ ফলাফল মোটামুটি নির্ভুল হওয়া উচিত। দ্বিতীয় আরেকটি গবেষণার নেতৃত্ব দিয়েছেন যুক্তরাজ্যের মেডিকেল রিসার্চ কাউন্সিলের (এমআরসি) ইমিউনোলজিস্ট তাও দং। তিনি টি-সেল শনাক্তকরণে কাজ করেছেন। টি-সেল শনাক্তকরণের প্রক্রিয়া অবশ্য অ্যান্টিবডির মতো এত আলোচিত নয়। তবে সংক্রমণের বিরুদ্ধে লড়াই এবং দীর্ঘমেয়াদি সুরক্ষায় সমান গুরুত্বপূর্ণ ভূমিকা পালন করে। গবেষণাসংক্রান্ত নিবন্ধ প্রকাশিত হয়েছে ‘নেচার ইমিউনোলজি’ সাময়িকীতে। তাঁরা বলছেন, গবেষণার ক্ষেত্রে কোভিড-১৯ মৃদু সংক্রমণের শিকার ২৮ ব্যক্তির রক্তের নমুনা, ১৪ জন গুরুতর অসুস্থ ও ১৬ জন সুস্থ ব্যক্তির রক্তের নমুনা পরীক্ষা করেছেন। গবেষণা নিবন্ধে বলা হয়, সংক্রমিত ব্যক্তিদের ক্ষেত্রে টি-সেলের তীব্র প্রতিক্রিয়া তাঁরা দেখেছেন। এ ক্ষেত্রে মৃদু ও গুরুতর অসুস্থ ব্যক্তিদের ক্ষেত্রে প্রতিক্রিয়ার ভিন্নতা পাওয়া গেছে।"""

# Now you can use this in your inference code
print(sample_text)

‘ইকোনমিস্ট’ লিখেছে, অ্যান্টিবডির চার মাস স্থায়ী হওয়ার খবরটি দুই কারণে আনন্দের। অ্যান্টিবডি যত দিন পর্যন্ত শরীরে টিকবে, তত দিন সংক্রমণ থেকে সুরক্ষিত থাকা সম্ভব। অর্থাৎ, এমন এক টিকার প্রয়োজন হবে, যা অ্যান্টিবডির উত্পাদনকে প্ররোচিত করতে পারে এবং দীর্ঘস্থায়ী সুরক্ষা দিতে পারে। এগুলো খুঁজে বের করাও সহজ। এটি আভাস দেয়, ব্যাপক হারে অ্যান্টিবডি শনাক্তকরণ ফলাফল মোটামুটি নির্ভুল হওয়া উচিত। দ্বিতীয় আরেকটি গবেষণার নেতৃত্ব দিয়েছেন যুক্তরাজ্যের মেডিকেল রিসার্চ কাউন্সিলের (এমআরসি) ইমিউনোলজিস্ট তাও দং। তিনি টি-সেল শনাক্তকরণে কাজ করেছেন। টি-সেল শনাক্তকরণের প্রক্রিয়া অবশ্য অ্যান্টিবডির মতো এত আলোচিত নয়। তবে সংক্রমণের বিরুদ্ধে লড়াই এবং দীর্ঘমেয়াদি সুরক্ষায় সমান গুরুত্বপূর্ণ ভূমিকা পালন করে। গবেষণাসংক্রান্ত নিবন্ধ প্রকাশিত হয়েছে ‘নেচার ইমিউনোলজি’ সাময়িকীতে। তাঁরা বলছেন, গবেষণার ক্ষেত্রে কোভিড-১৯ মৃদু সংক্রমণের শিকার ২৮ ব্যক্তির রক্তের নমুনা, ১৪ জন গুরুতর অসুস্থ ও ১৬ জন সুস্থ ব্যক্তির রক্তের নমুনা পরীক্ষা করেছেন। গবেষণা নিবন্ধে বলা হয়, সংক্রমিত ব্যক্তিদের ক্ষেত্রে টি-সেলের তীব্র প্রতিক্রিয়া তাঁরা দেখেছেন। এ

In [ ]:
# Ensure the model is on CPU
model.to("cpu")

# Make sure the input_ids tensor is on the same device as the model (CPU in this case)
device = model.device  # This should now be 'cpu'
input_ids = tokenizer.encode(sample_text, return_tensors="pt", max_length=1024, truncation=True).to(device)

# Generate summary
outputs = model.generate(input_ids, max_length=200, num_beams=7, early_stopping=True)
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(summary)

বাংলাদেশে কোভিড-১৯ মৃদু সংক্রমণের শিকার ২৮ ব্যক্তির রক্তের নমুনা পরীক্ষা করেছেন।


In [ ]:
save_path = '/content/drive/MyDrive/bangla_text_summarizer/mt5_weights/'

# Make all model parameters contiguous
for param in model.parameters():
    if not param.is_contiguous():
        param.data = param.data.contiguous()

# Save the model and tokenizer
model.save_pretrained(f"{save_path}fine_tuned_mt5_summarization_v3")
tokenizer.save_pretrained(f"{save_path}fine_tuned_mt5_summarization_v3")

('/content/drive/MyDrive/bangla_text_summarizer/mt5_weights/fine_tuned_mt5_summarization_v3/tokenizer_config.json',
 '/content/drive/MyDrive/bangla_text_summarizer/mt5_weights/fine_tuned_mt5_summarization_v3/special_tokens_map.json',
 '/content/drive/MyDrive/bangla_text_summarizer/mt5_weights/fine_tuned_mt5_summarization_v3/spiece.model',
 '/content/drive/MyDrive/bangla_text_summarizer/mt5_weights/fine_tuned_mt5_summarization_v3/added_tokens.json')

In [ ]:
from google.colab import runtime
runtime.unassign()